<a href="https://colab.research.google.com/github/Nikhil4002-50-82/Churn-Prediction/blob/main/ChurnPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
from google.colab import files
uploaded=files.upload()

!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [80]:
!kaggle datasets download blastchar/telco-customer-churn

Dataset URL: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
License(s): copyright-authors
telco-customer-churn.zip: Skipping, found more recently modified local copy (use --force to force download)


In [81]:
!unzip telco-customer-churn.zip

Archive:  telco-customer-churn.zip
replace WA_Fn-UseC_-Telco-Customer-Churn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: WA_Fn-UseC_-Telco-Customer-Churn.csv  


In [82]:
!pip install ydata-profiling
import pandas as pd
from ydata_profiling import ProfileReport

data=pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
profile = ProfileReport(data,title="Profiling Report")
profile.to_file("report1.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 21/21 [00:02<00:00,  8.54it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [83]:
y=data["Churn"]
X=data.drop(columns=["customerID","Churn"])

In [84]:
print(X.info())
print(X.isna().sum())
print(f"shape of the data : {X.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [85]:
y.value_counts()

,count
Churn,
No,5174
Yes,1869


In [86]:
y.sample(5)

,Churn
6785,Yes
4381,Yes
4540,No
2517,Yes
4029,No


In [87]:
X.sample()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
5626,Female,0,No,No,11,Yes,No,DSL,Yes,No,Yes,No,No,Yes,Month-to-month,No,Bank transfer (automatic),64.05,733.95


In [88]:
import numpy as np
X["TotalCharges"]=X["TotalCharges"].replace(" ",np.nan)
X["TotalCharges"]=X["TotalCharges"].astype(float)

In [89]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [90]:
X.isna().sum()

,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0
OnlineBackup,0


In [91]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [92]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

In [93]:
from sklearn import set_config
set_config(display="diagram")
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler

numericFeatures=[4,17,18]
categoricalFeatures=[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16]

# for i in numericFeatures:
#     print(i, X_train.columns[i], X_train.dtypes[i])
# print()
# for i in categoricalFeatures:
#     print(i, X_train.columns[i], X_train.dtypes[i])

categoricalPipeline=Pipeline([
    ("encoder",OneHotEncoder(drop="first",sparse_output=False))
])
numericalPipeline=Pipeline([
    ("imputer",SimpleImputer(strategy="mean")),
    ("scaler",StandardScaler())
])

transformer=ColumnTransformer([
    ("categorical",categoricalPipeline,categoricalFeatures),
    ("numerical",numericalPipeline,numericFeatures)
])

pipeline=Pipeline([
    ("preprocessor",transformer)
])

X_train=pipeline.fit_transform(X_train)
X_test=pipeline.transform(X_test)

featureNames=pipeline.named_steps["preprocessor"].get_feature_names_out()
X_train=pd.DataFrame(X_train,columns=featureNames)
X_test=pd.DataFrame(X_test,columns=featureNames)
y_train=pd.DataFrame(y_train,columns=["Churn"])
y_test=pd.DataFrame(y_test,columns=["Churn"])

pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse_output=False))]),
                                                  [0, 1, 2, 3, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15,
                                                   16]),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [4, 17, 18])]))])

In [94]:
X_train.head()

,categorical__gender_Male,categorical__SeniorCitizen_1,categorical__Partner_Yes,categorical__Dependents_Yes,categorical__PhoneService_Yes,categorical__MultipleLines_No phone service,categorical__MultipleLines_Yes,categorical__InternetService_Fiber optic,categorical__InternetService_No,categorical__OnlineSecurity_No internet service,...,categorical__StreamingMovies_Yes,categorical__Contract_One year,categorical__Contract_Two year,categorical__PaperlessBilling_Yes,categorical__PaymentMethod_Credit card (automatic),categorical__PaymentMethod_Electronic check,categorical__PaymentMethod_Mailed check,numerical__tenure,numerical__MonthlyCharges,numerical__TotalCharges
0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.881078,0.195927,0.653404
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.284263,0.522755,-0.976492
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.793997,-1.509551,-0.897371
3,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,-0.344587,1.053643,-0.012148
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.079985,0.308740,-0.812882


In [95]:
y_train.head()

,Churn
0,0
1,0
2,0
3,0
4,0


In [96]:
print(X_train.shape)
print(y_train.shape)

(4930, 30)
(4930, 1)


In [97]:
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [98]:
model.fit(X_train,y_train,epochs=100)

Epoch 1/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6383 - loss: 0.5922
Epoch 2/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7916 - loss: 0.4459
Epoch 3/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7961 - loss: 0.4401
Epoch 4/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7874 - loss: 0.4349
Epoch 5/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7926 - loss: 0.4312
Epoch 6/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7856 - loss: 0.4313
Epoch 7/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8055 - loss: 0.4274
Epoch 8/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8065 - loss: 0.4164
Epoch 9/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7979 - loss: 0.4210
Epoch 10/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8017 - loss: 0.4306
Epoch 11/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7964 - loss: 0.4310
Epoch 12/100
155/155 ━━━━━━━━━━━━━━━━━━━━

In [99]:
report2=ProfileReport(X_train,title="Profiling Report For Training Data After Preprocessing")
report2.to_file("report2.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 30/30 [00:01<00:00, 20.91it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [113]:
pred=pd.DataFrame(model.predict(X_test),columns=["PredictedValue"])
y_test=y_test.rename(columns={"Churn":"ActualValue"})
result=pd.concat([y_test,pred],axis=1)
result.sample(10)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,ActualValue,PredictedValue
845,0,0.240194
306,0,0.001742
37,1,0.568606
1176,0,0.036238
744,1,0.932728
105,1,0.229955
339,0,0.024578
763,1,0.561849
1246,1,0.662033
1668,0,0.037640
